https://supabase.com/blog/openai-embeddings-postgres-vector


In [3]:
!pip install supabase pgvector vecs openai -q

In [5]:
!pip install python-dotenv -q

from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file


True

In [1]:
from supabase import create_client, Client
import os

url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(url, key)

In [35]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = "gpt-4.1-mini"

# Function to get the vector embedding for a given text
def get_vector_embeddings(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"  # Using latest embedding model
    )
    embeddings = response.data[0].embedding
    return embeddings


query_embedding = get_vector_embeddings("Your text string goes here")
print(len(query_embedding))
query_embedding

1536


[0.005132982973009348,
 0.017242904752492905,
 -0.018698474392294884,
 -0.01855851523578167,
 -0.047250036150217056,
 -0.03028704971075058,
 0.027669822797179222,
 0.003614431945607066,
 0.011217685416340828,
 0.006434598471969366,
 -0.0016865016659721732,
 0.01581532694399357,
 -0.0012849952327087522,
 -0.00785867776721716,
 0.05990229919552803,
 0.050357118248939514,
 -0.02754385955631733,
 0.00991606991738081,
 -0.040392063558101654,
 0.04999322444200516,
 -0.0003859796852339059,
 0.030231066048145294,
 -0.013729942962527275,
 0.032974254339933395,
 0.01731288433074951,
 0.01680903322994709,
 -0.0017661030869930983,
 0.020391974598169327,
 0.040811941027641296,
 -0.03770485892891884,
 -0.02617226541042328,
 -0.0500212162733078,
 0.024184852838516235,
 -0.05519968643784523,
 -0.03224647045135498,
 0.04232349246740341,
 0.06466089189052582,
 0.014695657417178154,
 -0.015619384124875069,
 -0.04134378209710121,
 0.02218344435095787,
 0.007361824624240398,
 0.04495471343398094,
 0.007074

```
CREATE TABLE items (
    id serial PRIMARY KEY,
    embedding vector(1536),
    text TEXT
);
```


In [9]:
employee_handbook = """
Welcome to the "Unicorn Enterprises: Where Magic Happens" Employee Handbook! We're thrilled to have you join our team of dreamers, doers, and unicorn enthusiasts. At Unicorn Enterprises, we believe that work should be as enchanting as it is productive. This handbook is your ticket to the magical world of our company, where we'll outline the principles, policies, and practices that guide us on this extraordinary journey. So, fasten your seatbelts and get ready to embark on an adventure like no other!

**1: Our Magical Culture**
At Unicorn Enterprises, we take pride in our unique and enchanting company culture. We believe that creativity and innovation flourish best when people are happy and inspired. From our weekly "Wear Your Favorite Mythical Creature Costume" day on Fridays to our in-house unicorn petting zoo, we aim to infuse magic into every corner of our workplace. So, don't be surprised if you find a fairy tale book in the breakroom or a gnome guiding you to the restroom. Our culture is designed to spark your imagination and encourage collaboration among our magical team.

**2: Unicorn Code of Conduct**
While we embrace creativity, we also value professionalism. Our Unicorn Code of Conduct ensures that we maintain a harmonious and respectful environment. Treating all team members, regardless of their unicorn species, with kindness and respect is essential. We also encourage open communication and constructive feedback because, in our world, every opinion matters, just like every horn on a unicorn's head!

**3: Magical Work-Life Balance**
At Unicorn Enterprises, we understand the importance of maintaining a balanced life. We offer flexible work hours, magical mental health days, and even an on-site wizard to provide stress-relief spells when needed. We believe that a happy and well-rested employee is a creative and productive employee. So, don't hesitate to use our relaxation chambers or join a group meditation session under the office rainbow.

**4: Enchanted Benefits**
Our commitment to your well-being extends to our magical benefits package. You'll enjoy a treasure chest of perks, including unlimited unicorn rides, a bottomless cauldron of coffee and potions, and access to our company library filled with spellbinding books. We also offer competitive health and dental plans, ensuring your physical well-being is as robust as your magical spirit.

**5: Continuous Learning and Growth**
At Unicorn Enterprises, we believe in continuous learning and growth. We provide access to a plethora of online courses, enchanted workshops, and wizard-led training sessions. Whether you're aspiring to master new spells or conquer new challenges, we're here to support your personal and professional development.
As we conclude this handbook, remember that at Unicorn Enterprises, the pursuit of excellence is a never-ending quest. Our company's success depends on your passion, creativity, and commitment to making the impossible possible. We encourage you to always embrace the magic within and outside of work, and to share your ideas and innovations to keep our enchanted journey going. Thank you for being a part of our mystical family, and together, we'll continue to create a world where magic and business thrive hand in hand!
"""

# Split the employee handbook into paragraphs
chunks = [p.strip() for p in employee_handbook.split('\n\n') if p.strip()]
print(f"Number of paragraphs: {len(chunks)}")

# Print each paragraph with its index
for i, p in enumerate(chunks):
    print(f"\nParagraph {i}:")
    print(p)


Number of paragraphs: 6

Paragraph 0:
Welcome to the "Unicorn Enterprises: Where Magic Happens" Employee Handbook! We're thrilled to have you join our team of dreamers, doers, and unicorn enthusiasts. At Unicorn Enterprises, we believe that work should be as enchanting as it is productive. This handbook is your ticket to the magical world of our company, where we'll outline the principles, policies, and practices that guide us on this extraordinary journey. So, fasten your seatbelts and get ready to embark on an adventure like no other!

Paragraph 1:
**1: Our Magical Culture**
At Unicorn Enterprises, we take pride in our unique and enchanting company culture. We believe that creativity and innovation flourish best when people are happy and inspired. From our weekly "Wear Your Favorite Mythical Creature Costume" day on Fridays to our in-house unicorn petting zoo, we aim to infuse magic into every corner of our workplace. So, don't be surprised if you find a fairy tale book in the breakr

In [10]:
# Get embeddings for each chunk
embeddings = []
for chunk in chunks:
    embedding = get_vector_embeddings(chunk)
    embeddings.append(embedding)

# Insert chunks and their embeddings into Supabase
for chunk, embedding in zip(chunks, embeddings):
    data = supabase.table('items').insert({
        "text": chunk,
        "embedding": embedding
    }).execute()
    
print("Successfully inserted all chunks into Supabase")


Successfully inserted all chunks into Supabase


```
-- Ensure the pgvector extension is enabled
create extension if not exists vector;

-- Function to search for similar items
create or replace function match_items (
  query_embedding vector(1536), -- Match the dimension of your embeddings
  match_threshold float,        -- Minimum similarity score (e.g., 0.7)
  match_count int               -- Max number of results to return
)
returns table (
  id bigint,                  -- Match the type of your 'id' column
  text text,                  -- Match the type of your 'text' column
  similarity float            -- Calculated similarity score
)
language sql stable
as $$
  select
    items.id,
    items.text,
    1 - (items.embedding <=> query_embedding) as similarity -- '<=>' is cosine distance, 1 - distance = similarity
  from items
  where 1 - (items.embedding <=> query_embedding) > match_threshold
  order by items.embedding <=> query_embedding -- Order by distance (ascending)
  limit match_count;
$$;
```


```

In [42]:
# Your query text
query_text = "Do we get free Unicorn rides?"

# Generate the embedding for the query text
query_embedding = get_vector_embeddings(query_text)

# Call the database function using rpc
try:
    results = supabase.rpc('match_items', {
        'query_embedding': query_embedding,
        'match_threshold': 0.4,  # Adjust this threshold as needed
        'match_count': 2         # Number of results you want
    }).execute()

    # Process the results
    # Print each result with its similarity score
    for result in results.data:
        print(f"\nDocument ID: {result['id']}")
        print(f"Similarity Score: {result['similarity']:.3f}")
        print("Text:")
        print(result['text'])
        print("-" * 80)

except Exception as e:
    print(f"An error occurred: {e}")
    # Check if the error message gives clues, e.g., function not found



Document ID: 2
Similarity Score: 0.503
Text:
**1: Our Magical Culture**
At Unicorn Enterprises, we take pride in our unique and enchanting company culture. We believe that creativity and innovation flourish best when people are happy and inspired. From our weekly "Wear Your Favorite Mythical Creature Costume" day on Fridays to our in-house unicorn petting zoo, we aim to infuse magic into every corner of our workplace. So, don't be surprised if you find a fairy tale book in the breakroom or a gnome guiding you to the restroom. Our culture is designed to spark your imagination and encourage collaboration among our magical team.
--------------------------------------------------------------------------------

Document ID: 5
Similarity Score: 0.494
Text:
**4: Enchanted Benefits**
Our commitment to your well-being extends to our magical benefits package. You'll enjoy a treasure chest of perks, including unlimited unicorn rides, a bottomless cauldron of coffee and potions, and access to our

In [43]:
def retrieve(query_text, match_count=2):
    # Get vector embedding for the query text
    query_embedding = get_vector_embeddings(query_text)
    
    # Query using pgvector's cosine distance search
    response = supabase.rpc(
        'match_items',  # Use vector similarity search
        {
            'query_embedding': query_embedding,
            'match_threshold': 0.4,  # Similarity threshold
            'match_count': match_count  # Number of matches to return
        }
    ).execute()
    
    # Process and format results
    results = []
    for result in response.data:
        results.append({
            'id': result['id'],
            'similarity': result['similarity'],
            'text': result['text']
        })
    
    return results


query = "Do we get free Unicorn rides?"
retrieve(query)

[{'id': 2,
  'similarity': 0.502970903226465,
  'text': '**1: Our Magical Culture**\nAt Unicorn Enterprises, we take pride in our unique and enchanting company culture. We believe that creativity and innovation flourish best when people are happy and inspired. From our weekly "Wear Your Favorite Mythical Creature Costume" day on Fridays to our in-house unicorn petting zoo, we aim to infuse magic into every corner of our workplace. So, don\'t be surprised if you find a fairy tale book in the breakroom or a gnome guiding you to the restroom. Our culture is designed to spark your imagination and encourage collaboration among our magical team.'},
 {'id': 5,
  'similarity': 0.493960173017949,
  'text': "**4: Enchanted Benefits**\nOur commitment to your well-being extends to our magical benefits package. You'll enjoy a treasure chest of perks, including unlimited unicorn rides, a bottomless cauldron of coffee and potions, and access to our company library filled with spellbinding books. We a

In [46]:
def search_and_chat(chat_prompt, k=2):
    # Print the user query
    print("\nUser Query:")
    print(chat_prompt)
    print("-" * 80)
    
    # Perform the vector search using retrieve()
    search_results = retrieve(chat_prompt, k)
    
    # Print the retrieved context
    print("\nRetrieved Context:")
    for result in search_results:
        print(f"Document {result['id']} (Similarity: {result['similarity']:.3f})")
        print(result['text'])
        print()
    print("-" * 80)
    
    prompt_with_context = f"""Context: {search_results}\n\n
    Answer the question: {chat_prompt}"""

    # Create a list of messages for the chat
    messages = [
        {"role": "system", "content": "Please answer the questions provided by the user. Use only the context provided to you, if you don't know the answer say \"I don't know\"."},
        {"role": "user", "content": prompt_with_context},
    ]

    # Get the model's response
    response = client.chat.completions.create(
        model="gpt-4o-mini", # Using standard model since no MODEL constant defined
        messages=messages
    )

    # Print the model's response
    print("\nAI Response:")
    print(response.choices[0].message.content)
    print("-" * 80)

# Example usage
search_and_chat("Do we get free Unicorn rides?")



User Query:
Do we get free Unicorn rides?
--------------------------------------------------------------------------------

Retrieved Context:
Document 2 (Similarity: 0.503)
**1: Our Magical Culture**
At Unicorn Enterprises, we take pride in our unique and enchanting company culture. We believe that creativity and innovation flourish best when people are happy and inspired. From our weekly "Wear Your Favorite Mythical Creature Costume" day on Fridays to our in-house unicorn petting zoo, we aim to infuse magic into every corner of our workplace. So, don't be surprised if you find a fairy tale book in the breakroom or a gnome guiding you to the restroom. Our culture is designed to spark your imagination and encourage collaboration among our magical team.

Document 5 (Similarity: 0.494)
**4: Enchanted Benefits**
Our commitment to your well-being extends to our magical benefits package. You'll enjoy a treasure chest of perks, including unlimited unicorn rides, a bottomless cauldron of cof